# **MIE1624 Group 19 Course Project**

## **Prep**

### Import Packages and get data file ready

In [1]:
!python -m spacy download en_core_web_sm > /dev/null 2>&1

In [2]:
# You may need to restart runtime after the installation by the instruction
!pip install spacytextblob > /dev/null 2>&1


In [3]:
!pip install tqdm_notebook > /dev/null 2>&1

In [4]:
import os
import pandas as pd
import numpy as np
from sys import platform
import spacy
import html
import re
from spacytextblob.spacytextblob import SpacyTextBlob

def is_colab():
    return 'google.colab' in str(get_ipython())

if is_colab():
  from tqdm.notebook import tqdm_notebook as tqdm
else:
  from tqdm import tqdm

tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
# train_zip = 'train.zip'
# test_zip  = 'test.zip'
# train_file = 'train.csv'
# test_file = 'test.csv'

In [6]:
# def is_colab():
#     return 'google.colab' in str(get_ipython())

# def check_file(filename):
#     if not os.path.exists(filename):
#         if is_colab():
#             from google.colab import files
#             print(f"Please upload {filename} file")
#             uploaded = files.upload()
#         else:
#             print(f"{filename} not exist, this notebook require {filename} to run")
#     else:
#         print(f"{filename} ready")

# def unzip(src, dest):
#     if not os.path.exists(dest):
#         if platform == "linux" or platform == "linux2":
#             !unzip {src} 
#         elif platform == "win32":
#             !tar -xf {src}
#         else:
#             print(f"Please unzip {src} manually")

# check_file(train_zip)
# check_file(test_zip)

# # System check
# # Unzip data files if not already exist
# unzip(train_zip, train_file)
# unzip(test_zip, test_file)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

train_file = "/content/drive/MyDrive/Colab Notebooks/train.csv"
test_file = "/content/drive/MyDrive/Colab Notebooks/test.csv"
cleaned_train_file = "/content/drive/MyDrive/Colab Notebooks/cleaned_train.csv"
final_data = '/content/drive/MyDrive/Colab Notebooks/data.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Visualize Data

In [8]:
train_df, test_df = pd.read_csv(train_file), pd.read_csv(test_file)
train_df.head()

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,Jazz,$8.40,p23649501,3856620,NaN
1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,Alternative Rock,$10.98,p58458313,56086781,NaN
2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,Pop,$5.99,p97027626,55852154,NaN
3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,Jazz,$14.64,p43167086,43228100,NaN
4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,Alternative Rock,$9.92,p94494236,54425467,NaN


In [9]:
test_df.head()

,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,"02 11, 2017",u04140621,ok,Five Stars,1486771200,Pop,$31.20,p65721979,98944233,NaN
1,"01 29, 2018",u74766187,The 1955 and 1958 sets alone are worth the pur...,Buy It For the 1955 Set With Monk!,1517184000,Jazz,$34.04,p87809670,33903910,NaN
2,"08 29, 2005",u31689638,Mary J Blige is someone most people know of. S...,Mary's excellent debut! Classic!,1125273600,Pop,$1.72,p52702240,35386470,NaN
3,"11 2, 2015",u35613516,Very good satch album,Four Stars,1446422400,Pop,$10.99,p01314374,12404766,NaN
4,"01 3, 2012",u93320378,Well there are hundreds of Christmas anthologi...,CD lives up to its name,1325548800,Alternative Rock,$7.98,p84396269,68253688,NaN


In [10]:
print(f'training set shape: {train_df.shape}')
print(f'testing set shape: {test_df.shape}')

training set shape: (150000, 11)
testing set shape: (20000, 10)


In [11]:
print(f'Number of reviewText nan in training set: {train_df.reviewText.isna().sum()}')
print(f'Number of reviewText nan in testing set: {test_df.reviewText.isna().sum()}')

Number of reviewText nan in training set: 28
Number of reviewText nan in testing set: 3


## **Data Cleaning**

### Preprocess reviewText and summary using NLP

In [ ]:
train_df.dropna(axis=0, subset=['reviewText'], inplace=True)
test_df.dropna(axis=0, subset=['reviewText'], inplace=True)

#### Remove characters and urls


In [ ]:
def replace_character_codes(text):
    text = html.unescape(text)
    return text

def remove_url(text):
    text =  re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", "", text)
    return text


data_cleaning_funcs = [remove_url, replace_character_codes]

#### nlp sentiment analysis


In [ ]:
nlp = nlp = spacy.load('en_core_web_sm')
sentencizer = nlp.create_pipe("sentencizer")
spacy_text_blob = SpacyTextBlob()
nlp.add_pipe(spacy_text_blob)
nlp.add_pipe(sentencizer)
all_stopwords = nlp.Defaults.stop_words


def sentiment_analysis(text, use_tag=True):
  remove_non_alphanumeric_char = lambda _: re.sub(r'[^a-zA-Z]+', ' ', _).lower().strip()
  if len(text.strip()) > 0:
    doc = nlp(text)
    # Polarity, Subjectivity
    polarity, subjectivity = doc._.sentiment.polarity, doc._.sentiment.subjectivity
    
    sent = next(doc.sents)
    lemmatized_text = sent.lemma_.split()
    cleaned_lemmatized_text = [remove_non_alphanumeric_char(lemma) for lemma in lemmatized_text]
    if use_tag:
      tags = [token.tag_ for token in sent]
      
      wordsList = [lemma + '/' + tag 
                   for lemma, tag in zip(cleaned_lemmatized_text, tags)
                   if (not lemma in all_stopwords) and lemma]
    else:
      wordsList = [lemma
                   for lemma in cleaned_lemmatized_text  
                   if (not lemma in all_stopwords) and lemma]
    text = " ".join(wordsList)
    return pd.Series([polarity, subjectivity, text])
  else:
    return pd.Series([0, 0, text])

def process(df, use_tag=True):
  df['reviewText_cleaned'] = df.reviewText
  df['summary_cleaned'] = df.summary

  for func in data_cleaning_funcs:
    df['reviewText_cleaned'] = df.reviewText_cleaned.progress_apply(func)
    df['summary_cleaned'] = df.summary_cleaned.progress_apply(func)

  df[['reviewText_polarity', 'reviewText_subjectivity', 'reviewText_cleaned']] = df.reviewText_cleaned.progress_apply(sentiment_analysis)
  df[['summary_polarity', 'summary_subjectivity', 'summary_cleaned']] = df.summary_cleaned.progress_apply(sentiment_analysis)
  

# sentiment_analysis('', False)
process(train_df, use_tag=True)

### Turn price into float, review time keep only year

In [ ]:
def handle_price(price):
  try:
    return float(re.sub('[$]+', '', price))
  except:
    return -1

In [ ]:
train_df['reviewTime_year'] = train_df.reviewTime.progress_apply(lambda _: _[-4:])
train_df['price_float'] = train_df.price.progress_apply(handle_price)

train_df

,Unnamed: 0,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image,reviewText_cleaned,summary_cleaned,reviewText_polarity,reviewText_subjectivity,summary_polarity,summary_subjectivity,reviewTime_year,price_float
0,0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,Jazz,$8.40,p23649501,3856620,,contemporary/JJ jazz/NN soul/NN performer/JJ d...,classique/NNP,0.229849,0.426437,0.000000,0.000000,2014,8.40
1,1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,Alternative Rock,$10.98,p58458313,56086781,,good/JJ idea/NN pop/NN orchestral/JJ music/NN ...,exceed/VBD pron/PRP$ expectation/NNS album/NN ...,0.280937,0.460303,0.000000,0.000000,2003,10.98
2,2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,Pop,$5.99,p97027626,55852154,,great/JJ collection/NN carole/NNP king/NNP s/P...,carole/NNP king/NNP fans/NNPS,0.800000,0.750000,0.000000,0.000000,2017,5.99
3,3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,Jazz,$14.64,p43167086,43228100,,album/NN brilliant/JJ piece/NN jazz/NNP fusion...,master/NNP piece/NN,0.680208,0.800000,0.000000,0.000000,2013,14.64
4,4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,Alternative Rock,$9.92,p94494236,54425467,,wait/VBD long/JJ time/NN dvd/NN release/VBN,especially/RB pron/PRP like/VBP concert/NN vid...,0.129478,0.440930,0.000000,1.000000,2015,9.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149967,149995,5.0,"09 12, 2013",u29136602,There is just something about Wesley's voice t...,Just right.,1378944000,Alternative Rock,$9.49,p26326919,90575108,,wesley/NNP s/POS voice/NN pron/PRP swoon/NN li...,right/RB,0.195663,0.553061,0.285714,0.535714,2013,9.49
149968,149996,5.0,"05 15, 2016",u51715193,How many recordings of Verdis Requiem do you o...,Being there in vibrant immediacy of performan...,1463270400,Classical,$14.83,p69247882,74425587,,recording/NNS verdis/NNP requiem/NNP pron/PRP,vibrant/IN immediacy/JJ performance/IN spiritu...,0.263781,0.590520,0.083333,0.233333,2016,14.83
149969,149997,5.0,"06 13, 2004",u28696060,....their best overall album. After the revis...,Creative rebirth and possibly....,1087084800,Alternative Rock,$18.86,p07028930,20399056,,pron/PRP$ good/JJS overall/JJ album/NN,creative/JJ rebirth/NN possibly/RB,0.171429,0.457143,0.250000,1.000000,2004,18.86
149970,149998,5.0,"06 27, 2012",u04925906,"Believe it or not, once upon a time, artists l...",Once Upon A Time,1340755200,Pop,$19.99,p38606785,5931588,,believe/VB pron/PRP time/NN artist/NNS like/IN...,time/NN,0.095939,0.482133,0.000000,0.000000,2012,19.99


In [92]:
train_df.to_csv("/content/drive/MyDrive/Colab Notebooks/cleaned_train.csv", index=True)

## **Feature Engineering**

### Load cleaned train/test data

In [9]:
# save the preprocessed df in local
train_df = pd.read_csv(cleaned_train_file, index_col=0)
train_df.head()

,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image,reviewText_cleaned,summary_cleaned,reviewText_polarity,reviewText_subjectivity,summary_polarity,summary_subjectivity,reviewTime_year,price_float
0,5.0,"08 26, 2014",u92990698,A contemporary jazz and soul performer who's m...,""" CLASSIQUE """,1409011200,Jazz,$8.40,p23649501,3856620,,contemporary/JJ jazz/NN soul/NN performer/JJ d...,classique/NNP,0.229849,0.426437,0.0,0.0,2014,8.40
1,5.0,"05 11, 2003",u36200649,Very good idea to put both the 'pop' and 'orch...,Exceeded my Expectations - This album RAWKS!,1052611200,Alternative Rock,$10.98,p58458313,56086781,,good/JJ idea/NN pop/NN orchestral/JJ music/NN ...,exceed/VBD pron/PRP$ expectation/NNS album/NN ...,0.280937,0.460303,0.0,0.0,2003,10.98
2,5.0,"12 5, 2017",u10721702,This is a great collection of Carole King's so...,A Must-have for Carole King Fans,1512432000,Pop,$5.99,p97027626,55852154,,great/JJ collection/NN carole/NNP king/NNP s/P...,carole/NNP king/NNP fans/NNPS,0.800000,0.750000,0.0,0.0,2017,5.99
3,5.0,"09 24, 2013",u86003775,The is album is a brilliant piece of Jazz fusi...,A Master piece!,1379980800,Jazz,$14.64,p43167086,43228100,,album/NN brilliant/JJ piece/NN jazz/NNP fusion...,master/NNP piece/NN,0.680208,0.800000,0.0,0.0,2013,14.64
4,5.0,"01 14, 2015",u25672859,Waited a LONG time for this DVD to be released...,especially if you like concert videos,1421193600,Alternative Rock,$9.92,p94494236,54425467,,wait/VBD long/JJ time/NN dvd/NN release/VBN,especially/RB pron/PRP like/VBP concert/NN vid...,0.129478,0.440930,0.0,1.0,2015,9.92


In [10]:
train_df.dropna(inplace=True)
train_df.reviewText_cleaned.isnull().sum()

0

### Categorize data

In [11]:
data = train_df[[
                #  'reviewerID', 
                 'reviewTime_year', 
                 'category', 
                 'price_float', 
                 'reviewText_polarity', 
                 'reviewText_subjectivity', 
                 'summary_polarity', 
                 'summary_subjectivity',
                 'reviewText_cleaned',
                 'overall']]
# dummies_reviewerID = pd.get_dummies(data['reviewerID'], prefix='reviewerID')
dummies_category = pd.get_dummies(data['category'], prefix='category')
dummies_year = pd.get_dummies(data['reviewTime_year'], prefix='reviewTime_year')
data = pd.concat([data, dummies_year, dummies_category], axis=1)
data.drop(columns=['category', 'reviewTime_year'], inplace=True)
data.to_csv(final_data)

data

,price_float,reviewText_polarity,reviewText_subjectivity,summary_polarity,summary_subjectivity,reviewText_cleaned,overall,reviewTime_year_1998,reviewTime_year_1999,reviewTime_year_2000,reviewTime_year_2001,reviewTime_year_2002,reviewTime_year_2003,reviewTime_year_2004,reviewTime_year_2005,reviewTime_year_2006,reviewTime_year_2007,reviewTime_year_2008,reviewTime_year_2009,reviewTime_year_2010,reviewTime_year_2011,reviewTime_year_2012,reviewTime_year_2013,reviewTime_year_2014,reviewTime_year_2015,reviewTime_year_2016,reviewTime_year_2017,reviewTime_year_2018,category_Alternative Rock,category_Classical,category_Dance & Electronic,category_Jazz,category_Pop
0,8.40,0.229849,0.426437,0.000000,0.000000,contemporary/JJ jazz/NN soul/NN performer/JJ d...,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,10.98,0.280937,0.460303,0.000000,0.000000,good/JJ idea/NN pop/NN orchestral/JJ music/NN ...,5.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,5.99,0.800000,0.750000,0.000000,0.000000,great/JJ collection/NN carole/NNP king/NNP s/P...,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,14.64,0.680208,0.800000,0.000000,0.000000,album/NN brilliant/JJ piece/NN jazz/NNP fusion...,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,9.92,0.129478,0.440930,0.000000,1.000000,wait/VBD long/JJ time/NN dvd/NN release/VBN,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149967,9.49,0.195663,0.553061,0.285714,0.535714,wesley/NNP s/POS voice/NN pron/PRP swoon/NN li...,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
149968,14.83,0.263781,0.590520,0.083333,0.233333,recording/NNS verdis/NNP requiem/NNP pron/PRP,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
149969,18.86,0.171429,0.457143,0.250000,1.000000,pron/PRP$ good/JJS overall/JJ album/NN,5.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
149970,19.99,0.095939,0.482133,0.000000,0.000000,believe/VB pron/PRP time/NN artist/NNS like/IN...,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


## **Models**

In [38]:
data = pd.read_csv(final_data, index_col=0)

data.dropna(inplace=True)
# y = data[['overall']]
# X = data.drop(columns=['overall'])
print(data.overall)

0         5.0
1         5.0
2         5.0
3         5.0
4         5.0
         ... 
149967    5.0
149968    5.0
149969    5.0
149970    5.0
149971    5.0
Name: overall, Length: 147007, dtype: float64


#### Simple Models

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVR
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse

import scipy
from sklearn.base import BaseEstimator, TransformerMixin

In [43]:
max_iter = 100
max_features=500
n_gram = 2
models = {
    # 'Logistic Regression': LogisticRegression,
    'Naive-Bayes': GaussianNB,
    'SVM': SVC,
    'Decision Tree': DecisionTreeClassifier,
    'Random Forest': RandomForestClassifier,
    'XGBoost': GradientBoostingClassifier
}

vectorizers = {
    'word frequency': CountVectorizer,
    'tfidf': TfidfVectorizer,
    'ngram': CountVectorizer(analyzer='word', ngram_range=(n_gram, n_gram))
}


def vectorize(df, vectorizer):
  vc_fit = vectorizer.fit_transform(df.reviewText_cleaned)
  reviewText_df = pd.DataFrame(data=vc_fit.toarray(), columns = vectorizer.get_feature_names())
  new_df = pd.concat([df.reset_index(), reviewText_df.reset_index()], axis=1)
  new_df.drop(columns=['reviewText_cleaned'], inplace=True)
  return new_df

def test_models(data,
                max_iter=max_iter, 
                max_features=max_features, 
                models=models, 
                vectorizers=vectorizers,
                verbose=False
                ):
  y = data.overall
  X = data.drop(columns=['overall'])
  for vectorizer_name, vectorizer in vectorizers.items():
    for model_name, model in models.items():
      try:
        m = model(max_iter=max_iter)
      except:
        m = model()
      new_X = vectorize(X, vectorizer(max_features=max_features))
      
      X_train, X_test, y_train, y_test = train_test_split(new_X,
                                                    y,
                                                    test_size=0.3, 
                                                    random_state=42)
      m.fit(X_train, y_train)
      prediction = m.predict(X_test)
      print(np.sum(m.predict_proba(X_test), axis=1))
      error = mse(y_test, prediction)

      print(f'===== {model_name} =====')
      print(f'Mean Squared Error: {error}')

test_models(data)

[1. 1. 1. ... 1. 1. 1.]
===== Naive-Bayes =====
Mean Squared Error: 1.3088905516631522


KeyboardInterrupt: ignored

### Random Forest

